# Импорт библиотек

In [41]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Константы

In [42]:
DATA_PATH = "../data/"
SAVE_PATH = "../charts/"

START_DATE = datetime(2005, 7, 1)

# Загружаем данные
BETA = pd.read_excel(DATA_PATH + "beta_gibbs.xlsx", header=None).values
X_MAT = pd.read_excel(DATA_PATH + "X.xlsx", header=None).values
SHOCKS_AS = pd.read_excel(DATA_PATH + "ETA_record AS.xlsx", header=None).values
SHOCKS_PRIVATED_AD = pd.read_excel(DATA_PATH + "ETA_record PrivateAD.xlsx", header=None).values
SHOCKS_MP = pd.read_excel(DATA_PATH + "ETA_record MP.xlsx", header=None).values
SHOCKS_FP = pd.read_excel(DATA_PATH + "ETA_record FP.xlsx", header=None).values
A0INV_P = pd.read_excel(DATA_PATH + "struct_irf_record for p.xlsx", header=None).values
A0INV_Y = pd.read_excel(DATA_PATH + "struct_irf_record for Y.xlsx", header=None).values
A0INV_I = pd.read_excel(DATA_PATH + "struct_irf_record for i.xlsx", header=None).values
A0INV_B = pd.read_excel(DATA_PATH + "struct_irf_record for B.xlsx", header=None).values

ACTUAL_P = pd.read_excel(DATA_PATH + 'actual_p.xlsx', header=None)[0].tolist()
ACTUAL_I = pd.read_excel(DATA_PATH + 'actual_i.xlsx', header=None)[0].tolist()

# Задаем параметры модели
T = X_MAT.shape[0]
N_ITER = BETA.shape[1]
N_VARS = 4
N_LAGS = 2
N_REG = 10

# Индексы переменных
P = 0
Y = 1
I = 2
B = 3
T_FIX = 73
I_TARGET = 7.5
LAST_POINT = T - T_FIX + 1

QUARTERS = [START_DATE + timedelta(days=91.25 * i) for i in range(T)]

# Готовим трехмерный объект для хранения 1000 контрфактических итераций по 4 переменным
CF = np.zeros((T, N_ITER, N_VARS))

In [43]:
def go_plot(title: str, name: str, file_name:str, quarters, median, low, high, actual):
    """Рисуем и сохраняем интерактивный go график.

    Args:
        title (str): Имя графика
        name (str): Имя ряда
        file_name (str): Имя файла, для сохранения
        quarters (_type_): Временная квартальная шкала
        p_median (_type_): Медианное значение
        p_low (_type_): Нижний ДИ
        p_high (_type_): Верхний ДИ
    """
    # Создаем фигуру
    fig = go.Figure()

    # Добавляем линии на график
    fig.add_trace(go.Scatter(x=quarters[-LAST_POINT:], y=median[-LAST_POINT:], mode='lines', name=name, line=dict(color='blue', width=3)))
    
    fig.add_trace(go.Scatter(x=quarters[-LAST_POINT:], y=low[-LAST_POINT:], mode='lines',
                             name='68% ДИ: нижняя', line=dict(color='blue', width=1, dash='dash')))

    fig.add_trace(go.Scatter(x=quarters[-LAST_POINT:], y=high[-LAST_POINT:], mode='lines',
                             name='68% ДИ: верхняя', line=dict(color='blue', width=1, dash='dash')))

    fig.add_trace(go.Scatter(x=quarters, y=actual, mode='lines',
                             name='Фактическая', line=dict(color='black', width=3)))
    
    # Настраиваем оформление
    fig.update_layout(title=title, xaxis_title='Квартал', yaxis_title='Инфляция (%)', plot_bgcolor='white', showlegend=True,
                      paper_bgcolor='white', legend=dict(bordercolor='black', borderwidth=1),
                      shapes=[dict(type="rect", xref="paper", yref="paper", x0=0, y0=0, x1=1, y1=1,
                                   line=dict(color="black", width=1))])

    # Добавляем сетку
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

    # Сохранить график
    fig.write_html(f"{SAVE_PATH}{file_name}.html")

# I. КОНТРФАКТИЧЕСКИЙ АНАЛИЗ

## 1) Фиксированная ключевая ставка (i) = 7,5

In [ ]:
# Готовим объект чтобы потом мочь посмотреть новые шоки ДКП
CF_EPS_MP = np.zeros((N_ITER, T))

for i in range(N_ITER):
    # Распаковываем параметры каждой итерации
    beta_i = BETA[:, i]
    
    # Восстанавливаем коэффициенты VAR
    B_p = beta_i[0:10]
    B_y = beta_i[10:20]
    B_i = beta_i[20:30]
    B_b = beta_i[30:40]
    
    # Структурная матрица A0^{-1} для каждой итерации
    A0inv_full = np.vstack([A0INV_P[i, :], A0INV_Y[i, :], A0INV_I[i, :],  A0INV_B[i, :]])
    
    # Шоки
    shocks_as = SHOCKS_AS[i, :].reshape(-1, 1)
    shocks_ad = SHOCKS_PRIVATED_AD[i, :].reshape(-1, 1)
    shocks_mp = SHOCKS_MP[i, :].reshape(-1, 1)
    shocks_fp = SHOCKS_FP[i, :].reshape(-1, 1)
    
    # Восстанавливаем начальные значения переменных до t_fix
    Y_temp = np.zeros((T, N_VARS))
    
    for t in range(T_FIX):
        x_t = X_MAT[t, :].reshape(-1, 1)
        eps_t = np.vstack([shocks_as[t], shocks_ad[t], shocks_mp[t], shocks_fp[t]])
        
        Y_temp[t, P] = (B_p.T @ x_t + A0inv_full[P, :] @ eps_t).item()
        Y_temp[t, Y] = (B_y.T @ x_t + A0inv_full[Y, :] @ eps_t).item()
        Y_temp[t, I] = (B_i.T @ x_t + A0inv_full[I, :] @ eps_t).item()
        Y_temp[t, B] = (B_b.T @ x_t + A0inv_full[B, :] @ eps_t).item()
    
    # Считаем контрфакт с момента фиксации
    for t in range(T_FIX, T):
        # Собираем лаги (размер (4,))
        l1 = Y_temp[t-1, :].reshape(-1, 1)
        l2 = Y_temp[t-2, :].reshape(-1, 1)

        # Формируем x_sim как вектор [10 x 1]
        x_sim = np.vstack([l1, l2, X_MAT[t, 8], X_MAT[t, 9]])
        
        # Вычисляем шоки
        eps_as = shocks_as[t]
        eps_ad = shocks_ad[t]
        eps_fp = shocks_fp[t]
        
        # Вычисляем eps_mp
        numerator = I_TARGET - (B_i.T @ x_sim + A0inv_full[I, [0, 1, 3]] @ np.vstack([eps_as, eps_ad, eps_fp]))
        eps_mp = (numerator / A0inv_full[I, 2]).item()
        
        # Собираем вектор шоков [4 x 1]
        eps_t = np.vstack([eps_as, eps_ad, eps_mp, eps_fp])
        
        # Обновляем Y_temp
        Y_temp[t, P] = (B_p.T @ x_sim + A0inv_full[P, :] @ eps_t).item()
        Y_temp[t, Y] = (B_y.T @ x_sim + A0inv_full[Y, :] @ eps_t).item()
        Y_temp[t, I] = I_TARGET  # фиксируем ставку
        Y_temp[t, B] = (B_b.T @ x_sim + A0inv_full[B, :] @ eps_t).item()
        
        # Обновляем шок MP
        shocks_mp[t] = eps_mp.item() if isinstance(eps_mp, np.ndarray) else eps_mp
    
    # Сохраняем результат симуляции
    CF[:, i, :] = Y_temp
    CF_EPS_MP[i, :] = shocks_mp.ravel()

# Анализируем
cf_p = CF[:, :, P]

p_median = np.nanmedian(cf_p, axis=1)
p_low = np.nanpercentile(cf_p, 16, axis=1)
p_high = np.nanpercentile(cf_p, 84, axis=1)

go_plot(title='Инфляция, QoQ: фактическая vs контрфактическая (КС = 7,5% с 3 квартала 2023 г.)',
        name='Контрфакт (КС = 7,5%)', file_name='I. КА. 1. Фиксированная ключевая ставка (i) = 7,5',
        quarters=QUARTERS, median=p_median, low=p_low, high=p_high, actual=ACTUAL_P)

## 2) Нулевые шоки ДКП

In [45]:
# Готовим объект чтобы потом мочь посмотреть новые шоки ДКП
CF_ZERO_MP = np.zeros((T, N_ITER, N_VARS))

for i in range(N_ITER):
    # Распаковываем параметры каждой итерации
    beta_i = BETA[:, i]
    
    # Восстанавливаем коэффициенты VAR
    B_p = beta_i[0:10]
    B_y = beta_i[10:20]
    B_i = beta_i[20:30]
    B_b = beta_i[30:40]
    
    # Структурная матрица A0^{-1} для каждой итерации
    A0inv_full = np.vstack([A0INV_P[i, :], A0INV_Y[i, :], A0INV_I[i, :],  A0INV_B[i, :]])
    
    # Шоки
    shocks_as = SHOCKS_AS[i, :].reshape(-1, 1)
    shocks_ad = SHOCKS_PRIVATED_AD[i, :].reshape(-1, 1)
    shocks_mp = SHOCKS_MP[i, :].reshape(-1, 1)
    shocks_fp = SHOCKS_FP[i, :].reshape(-1, 1)
    
    # Восстанавливаем начальные значения переменных до t_fix
    Y_temp = np.zeros((T, N_VARS))
    
    for t in range(T_FIX):
        x_t = X_MAT[t, :].reshape(-1, 1)
        eps_t = np.vstack([shocks_as[t], shocks_ad[t], shocks_mp[t], shocks_fp[t]])
        
        Y_temp[t, P] = (B_p.T @ x_t + A0inv_full[P, :] @ eps_t).item()
        Y_temp[t, Y] = (B_y.T @ x_t + A0inv_full[Y, :] @ eps_t).item()
        Y_temp[t, I] = (B_i.T @ x_t + A0inv_full[I, :] @ eps_t).item()
        Y_temp[t, B] = (B_b.T @ x_t + A0inv_full[B, :] @ eps_t).item()
    
    # С 2023Q3 задаём MP-шок = 0
    for t in range(T_FIX, T):
        # Собираем лаги (размер (4,))
        l1 = Y_temp[t-1, :].reshape(-1, 1)
        l2 = Y_temp[t-2, :].reshape(-1, 1)

        # Формируем x_sim как вектор [10 x 1]
        x_sim = np.vstack([l1, l2, X_MAT[t, 8], X_MAT[t, 9]])
        
        # Вычисляем шоки
        eps_as = shocks_as[t]
        eps_ad = shocks_ad[t]
        eps_mp_zero = 0  # фиксация
        eps_fp = shocks_fp[t]
        
        eps_zero = np.vstack([eps_as, eps_ad, eps_mp_zero, eps_fp])
        
        # Обновляем Y_temp
        Y_temp[t, P] = (B_p.T @ x_sim + A0inv_full[P, :] @ eps_zero).item()
        Y_temp[t, Y] = (B_y.T @ x_sim + A0inv_full[Y, :] @ eps_zero).item()
        Y_temp[t, I] = (B_i.T @ x_sim + A0inv_full[I, :] @ eps_zero).item()
        Y_temp[t, B] = (B_b.T @ x_sim + A0inv_full[B, :] @ eps_zero).item()
    
    # Сохраняем результат симуляции
    CF_ZERO_MP[:, i, :] = Y_temp

# Анализируем
cf_zeroMP_i = CF_ZERO_MP[:, :, I]
cf_zeroMP_p = CF_ZERO_MP[:, :, P]

p_median_zero = np.nanmedian(cf_zeroMP_p, axis=1)
p_low_zero = np.nanpercentile(cf_zeroMP_p, 16, axis=1)
p_high_zero = np.nanpercentile(cf_zeroMP_p, 84, axis=1)

# Строим графики. Инфляция
go_plot(title='Инфляция, QoQ: фактическая vs контрфактическая (шок ДКП = 0 с 3 квартала 2023 г.)',
        name='Контрфакт (шок ДКП = 0)', file_name='I. КА. 2. Нулевые шоки ДКП (Инфляция)',
        quarters=QUARTERS, median=p_median_zero, low=p_low_zero, high=p_high_zero, actual=ACTUAL_P)


i_median_zero = np.nanmedian(cf_zeroMP_i, axis=1)
i_low_zero = np.nanpercentile(cf_zeroMP_i, 16, axis=1)
i_high_zero = np.nanpercentile(cf_zeroMP_i, 84, axis=1)

# Строим графики. Ставка
go_plot(title='Среднеквартальная ключевая ставка: фактическая vs контрфактическая (шок ДКП = 0 с 3 квартала 2023 г.)',
        name='Контрфакт (шок ДКП = 0)', file_name='I. КА. 2. Нулевые шоки ДКП (Ставка)',
        quarters=QUARTERS, median=i_median_zero, low=i_low_zero, high=i_high_zero, actual=ACTUAL_I)